# 训练模型

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_train.ipynb)&emsp;[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_train.py)&emsp;[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/beginner/model_train.ipynb)

通过上面章节的学习，我们已经学会如何创建模型和构建数据集，现在开始学习如何设置超参和优化模型参数。

## 超参

超参是可以调整的参数，可以控制模型训练优化的过程，不同的超参数值可能会影响模型训练和收敛速度。一般会定义以下用于训练的超参：

- 训练轮次（epoch）：训练时遍历数据集的次数。
- 批次大小（batch size）：数据集进行分批读取训练，设定每个批次数据的大小。
- 学习率（learning rate）：如果学习率偏小，会导致收敛的速度变慢，如果学习率偏大则可能会导致训练不收敛等不可预测的结果。

In [19]:
epochs = 1
batch_size = 32
learning_rate = 1e-3

## 损失函数

**损失函数**用来评价模型的**预测值**和**真实值**之间的误差，在这里，使用绝对误差损失函数`L1Loss`。`mindspore.nn.loss`也提供了许多其他常用的损失函数，如`SoftmaxCrossEntropyWithLogits`、`MSELoss`、`SmoothL1Loss`等。

我们给定输出值和目标值，通过损失函数计算预测值和目标值之间的误差（损失值），使用方法如下所示：

In [20]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor

loss = nn.L1Loss()
output_data = Tensor(np.array([[1, 2, 3], [2, 3, 4]]).astype(np.float32))
target_data = Tensor(np.array([[0, 2, 5], [3, 1, 1]]).astype(np.float32))
print(loss(output_data, target_data))

1.5


## 优化器函数

优化器函数用于计算和更新梯度，模型优化算法的选择直接关系到最终模型的性能。有时候最终模型效果不好，未必是特征或者模型设计的问题，很有可能是优化算法的问题。MindSpore所有优化逻辑都封装在`Optimizer`对象中，在这里，我们使用`Momentum`优化器。`mindspore.nn`也提供了许多其他常用的优化器函数，如`Adam`、`SGD`、`RMSProp`等。

我们需要构建一个`Optimizer`对象，这个对象能够基于计算得到的梯度对参数进行更新。为了构建一个`Optimizer`，我们需要给它一个包含可优化的参数（必须是Variable对象）的迭代器，如网络中所有可以训练的`parameter`，将`params`设置为`net.trainable_params()`即可。然后，你可以设置`Optimizer`的参数选项，比如学习率、权重衰减等。

代码样例如下：

In [21]:
from mindspore import nn
from mindvision.classification.models import lenet

net = lenet(num_classes=10, pretrained=False)
optim = nn.Momentum(net.trainable_params(), 0.1, 0.9)

## 模型训练

模型训练一般分为四个步骤：

1. 构建数据集。
2. 定义神经网络。
3. 定义超参、损失函数及优化器。
4. 输入训练轮次和数据集进行训练。

模型训练示例代码如下：

In [ ]:
import mindspore.nn as nn
from mindspore import context
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

from mindvision.classification.dataset import Mnist
from mindvision.classification.models import lenet
from mindvision.engine.callback import LossMonitor

context.set_context(mode=context.GRAPH_MODE)

# 构建数据集
download_train = Mnist(path="./mnist", split="train", batch_size=32, repeat_num=1, shuffle=True, resize=32, download=True)
dataset_train = download_train.run()

# 定义神经网络
network = lenet(num_classes=10, pretrained=False)
# 定义损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
# 定义优化器函数
net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)

# 初始化模型参数
model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={'acc'})

# 设置模型保存参数
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
# 应用模型保存参数
ckpoint = ModelCheckpoint(prefix="lenet", directory="./lenet", config=config_ck)

model.train(epochs, dataset_train, callbacks=[ckpoint, LossMonitor(125)], dataset_sink_mode=False)